In [2]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
# %pip install openpyxl

  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: c:\users\ajit yadav\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip


In [31]:
# %pip install defusedxml 


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: c:\users\ajit yadav\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import xlrd
xlrd.__version__

'2.0.1'

In [260]:
from scipy.stats import norm
import numpy as np

class frtb_risk:
    def __init__(self, instument=None,risk_weight=None, bucket=None, position=None, T=None, r: float = None, S: float = None, K: float = None, sigma: float = None):
        self.instument = instument ##
        self.risk_weight=risk_weight
        self.bucket = bucket
        self.position = position
        self.S = S
        self.K = K
        self.sigma = sigma
        self.T = T
        self.r = r

    def cvr(self, option_type=None):
        S_values = np.array(self.S)
        risk_weight_value=np.array(self.risk_weight)
        K_values = np.array(self.K)
        r_values = np.array(self.r)
        T_values = np.array(self.T)
        sigma_values = np.array(self.sigma)
        types = option_type.values.astype(str)
        
        present_value = K_values * np.exp((-r_values) * T_values)
        
        cases=[S_values ,S_values * (1 + risk_weight_value), S_values * (1 -risk_weight_value )]
        results=[]
        d=[]
        
        for case in cases:

            d1= (np.log(case / K_values) + (r_values + 0.5 * sigma_values ** 2) * T_values) / (sigma_values * np.sqrt(T_values))
            d2= d1 - sigma_values * np.sqrt(T_values)


            option_price_call = case * norm.cdf(d1) - K_values * np.exp(-r_values * T_values) * norm.cdf(d2)
            option_price_put = K_values * np.exp(-r_values * T_values) * norm.cdf(-d2) - case * norm.cdf(-d1)

            value_instument = case * norm.cdf(d1) - present_value * norm.cdf(d2)
    
            results.append(value_instument)
            d.append((d1,d2))
            
            cvr_up = -(results[0][1]-results[0][0] - d[0][0] * (S_values*risk_weight_value))
            cvr_dn = -(results[0][0]-results[0][1] + d[0][0] * (S_values*risk_weight_value))

        return pd.DataFrame({'value':results[0],
                'value_up':results[0],
                'value_down':results[0],
                'cvr_up':cvr_up,
                'cvr_dn':cvr_dn} ,index= self.bucket)      
    def intra_bucket_correlations():
        
        pass
    
    def intra_bucket_correlations():
        
        pass

In [261]:
data=pd.read_excel('6404-3.1.4 Equity(FRTB-SA) CRUVATURE.xlsm',sheet_name='2.5').fillna(" ")
rf=data.iloc[2,2]
rf
# instuments_data
data
instuments_data=data.iloc[5:11,1:12]
instuments_data

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
5,Option,Stock,Bucket,RW,,Type,Position,Spot,T,Strike,sigma
6,1,CVS,5,0.3,0.25,Call,long,100,1,90,0.2
7,2,CVS,5,0.3,0.25,Call,short,50,2,50,0.15
8,3,Walmart,5,0.3,0.25,Call,long,150,3,155,0.18
9,4,IBM,8,0.5,0.25,Put,long,88,1,85,0.16
10,5,Bofa,8,0.5,0.25,Put,short,210,2,200,0.22


In [262]:
instuments_data.columns=instuments_data.loc[5]
instuments_data=instuments_data.loc[6:]
instuments_data.columns

Index(['Option', 'Stock', 'Bucket', 'RW', ' ', 'Type', 'Position', 'Spot', 'T',
       'Strike', 'sigma'],
      dtype='object', name=5)

In [263]:
instuments_data.rename(columns={' ':'p'},inplace=True)
instuments_data.columns=instuments_data.columns.str.lower()
# df['name'] = df['name'].apply(lambda x: x.lower() if isinstance(x, str) else x)
instuments_data

5,option,stock,bucket,rw,p,type,position,spot,t,strike,sigma
6,1,CVS,5,0.3,0.25,Call,long,100,1,90,0.2
7,2,CVS,5,0.3,0.25,Call,short,50,2,50,0.15
8,3,Walmart,5,0.3,0.25,Call,long,150,3,155,0.18
9,4,IBM,8,0.5,0.25,Put,long,88,1,85,0.16
10,5,Bofa,8,0.5,0.25,Put,short,210,2,200,0.22


In [264]:
dtype_conversions = {
    
    'option': int
     , 'stock': str
    , 'bucket': int
    , 'rw': float
    , 'p': float
    , 'type': str
    , 'position': str 
    , 'spot': float
    , 't': float
    ,'strike': float 
    ,'sigma' : float
}

# Apply the data type conversions to the DataFrame
instuments_data = instuments_data.astype(dtype_conversions)
instuments_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 6 to 10
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   option    5 non-null      int32  
 1   stock     5 non-null      object 
 2   bucket    5 non-null      int32  
 3   rw        5 non-null      float64
 4   p         5 non-null      float64
 5   type      5 non-null      object 
 6   position  5 non-null      object 
 7   spot      5 non-null      float64
 8   t         5 non-null      float64
 9   strike    5 non-null      float64
 10  sigma     5 non-null      float64
dtypes: float64(6), int32(2), object(3)
memory usage: 532.0+ bytes


In [265]:
instuments_data.index=instuments_data.bucket
instuments_data

5,option,stock,bucket,rw,p,type,position,spot,t,strike,sigma
bucket,,,,,,,,,,,
5,1,CVS,5,0.3,0.25,Call,long,100.0,1.0,90.0,0.20
5,2,CVS,5,0.3,0.25,Call,short,50.0,2.0,50.0,0.15
5,3,Walmart,5,0.3,0.25,Call,long,150.0,3.0,155.0,0.18
8,4,IBM,8,0.5,0.25,Put,long,88.0,1.0,85.0,0.16
8,5,Bofa,8,0.5,0.25,Put,short,210.0,2.0,200.0,0.22


In [266]:
equity=frtb_risk(instument=None,risk_weight=instuments_data.rw,
    bucket=instuments_data.bucket,position=instuments_data.position,
    T=instuments_data.t.values,r=rf,S=instuments_data.spot.values,
    K=instuments_data.strike.values,sigma=instuments_data.sigma.values,)

# import math
from scipy.stats import norm
option=equity.cvr(instuments_data.type)
option

,value,value_up,value_down,cvr_up,cvr_dn
bucket,,,,,
5,16.699448,16.699448,16.699448,36.165164,-36.165164
5,6.838362,6.838362,6.838362,18.523145,-18.523145
5,27.063448,27.063448,27.063448,33.793724,-33.793724
8,9.690869,9.690869,9.690869,36.669615,-36.669615
8,41.366512,41.366512,41.366512,76.409372,-76.409372


In [267]:
table=pd.concat([option,instuments_data.stock], axis=1)
table

,value,value_up,value_down,cvr_up,cvr_dn,stock
bucket,,,,,,
5,16.699448,16.699448,16.699448,36.165164,-36.165164,CVS
5,6.838362,6.838362,6.838362,18.523145,-18.523145,CVS
5,27.063448,27.063448,27.063448,33.793724,-33.793724,Walmart
8,9.690869,9.690869,9.690869,36.669615,-36.669615,IBM
8,41.366512,41.366512,41.366512,76.409372,-76.409372,Bofa


In [270]:
data_correl=pd.pivot_table(table,index=['stock','bucket'],aggfunc = np.sum)
data_correl

,,cvr_dn,cvr_up,value,value_down,value_up
stock,bucket,,,,,
Bofa,8,-76.409372,76.409372,41.366512,41.366512,41.366512
CVS,5,-54.688309,54.688309,23.537810,23.537810,23.537810
IBM,8,-36.669615,36.669615,9.690869,9.690869,9.690869
Walmart,5,-33.793724,33.793724,27.063448,27.063448,27.063448
